In [1]:
import os
import random
import time
import collections
import itertools
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pickle
import seaborn as sns
from functools import partial

from IPython.display import HTML
from scipy.stats import entropy
from innvariant_dgl.dataset import graph_to_construction_sequence, construction_sequence_to_graph, construction_sequence_evolution
from tqdm import tqdm

In [2]:
with open('deepgg-combined.pkl', 'rb') as read_handle:
    C = pickle.load(read_handle)

In [3]:
keys = list(C.keys())
keys_dfs = [key for key in keys if '_dfs' in key]
keys_bfs = [key for key in keys if '_bfs' in key]
keys_process = [key for key in keys if '_process' in key]
keys_erdos_renyi = [key for key in keys if 'erdos_renyi' in key]
keys_barabasi_albert = [key for key in keys if 'barabasi_albert' in key]
keys_watts_strogatz = [key for key in keys if 'watts_strogatz' in key]
keys_er_and_bfs = [key for key in keys if 'erdos_renyi' in key and '_bfs' in key]
keys_er_and_dfs = [key for key in keys if 'erdos_renyi' in key and '_dfs' in key]
keys_ba_and_bfs = [key for key in keys if 'barabasi_albert' in key and '_bfs' in key]
keys_ba_and_dfs = [key for key in keys if 'barabasi_albert' in key and '_dfs' in key]
keys_ws_and_bfs = [key for key in keys if 'watts_strogatz' in key and '_bfs' in key]
keys_ws_and_dfs = [key for key in keys if 'watts_strogatz' in key and '_dfs' in key]

subsets = {
    'Depth-First-Search': keys_dfs,
    'Breadth-First-Search': keys_bfs,
    'Construction Process': keys_process,
    'Erdos-Renyi model': keys_erdos_renyi,
    'Barabasi-Albert model': keys_barabasi_albert,
    'Watts-Strogatz model': keys_watts_strogatz,
    'Erdos-Renyi + BFS': keys_er_and_bfs,
    'Erdos-Renyi + DFS': keys_er_and_dfs,
    'Barabasi-Albert + BFS': keys_ba_and_bfs,
    'Barabasi-Albert + DFS': keys_ba_and_dfs,
    'Watts-Strogatz + BFS': keys_ws_and_bfs,
    'Watts-Strogatz + DFS': keys_ws_and_dfs,
}

In [6]:
print('Total number of instances: ', len(keys))
for subset_name in subsets:
    print('{name} has {num} instances'.format(name=subset_name, num=len(subsets[subset_name])))

Total number of instances:  69
Depth-First-Search has 39 instances
Breadth-First-Search has 24 instances
Construction Process has 6 instances
Erdos-Renyi model has 24 instances
Barabasi-Albert model has 26 instances
Watts-Strogatz model has 19 instances
Erdos-Renyi + BFS has 13 instances
Erdos-Renyi + DFS has 11 instances
Barabasi-Albert + BFS has 4 instances
Barabasi-Albert + DFS has 16 instances
Watts-Strogatz + BFS has 7 instances
Watts-Strogatz + DFS has 12 instances


In [18]:
C[list(C.keys())[0]]['meta']['params']

{'param_train_epochs': 8,
 'param_learning_rate': 0.0001,
 'param_deepgg_prop_rounds': 2,
 'param_deepgg_v_max': 150,
 'param_deepgg_node_hidden_size': 16,
 'param_deepgg_generate_size': 200,
 'param_deepgg_generate_v_min': 0,
 'param_dataset_size': 1000,
 'param_model_num_v': 50,
 'param_model_num_v_min': 50,
 'param_model_num_v_max': 50,
 'param_model_er_p': 0.2,
 'param_model_ws_k': 10,
 'param_model_ws_p': 0.2,
 'param_model_ba_m': 3,
 'param_selected_model': 'generate_seqs_watts_strogatz_dfs',
 'param_device': 'cuda:7'}

In [11]:
np.unique([C[key]['meta']['params']['param_model_er_p'] for key in keys_erdos_renyi])

array([0.2])

In [19]:
print('ws_k', np.unique([C[key]['meta']['params']['param_model_ws_k'] for key in keys_watts_strogatz]))
print('ws_p', np.unique([C[key]['meta']['params']['param_model_ws_p'] for key in keys_watts_strogatz]))

ws_k [10]
ws_p [0.2]


In [20]:
np.unique([C[key]['meta']['params']['param_model_ba_m'] for key in keys_barabasi_albert])

array([3])